# GOAL

so for this first implementation of a concept head, we're just gonna attempt to extend the context length of the model by giving the GPT a kind of recurrent hidden state that basically summarizes the previous length t window that it was looking at. to make this work imma have to train on the last context length chunk of the total sequence first, and then work backwards. each time i'll be passing the concept vector that was wanted to the previous chunk, and then regression / cosine similarity training on it. if i had a dataset other than tinyshakespeare that actually had \<endoftext> tokens then it'd be interesting to train that last context length chunk of the sequence to predict that token. Also I might need a learnable token to separate the concept from the rest of the sequence

a limitation of this approach is that it only works in chunks of the context length. if it works well then this is useful bc it means we could use a shorter context length which is important because attention is quadratic in its use of compute as a function of context length. however it's not clear whether the current approach will work with varying context lengths or if it'll only give us a useable context vector for the pre-determined context length t

yoooo so to speed up inference what if i ran a bunch of inferences on a bunch of sequences & got all the concept vectors that came out, then trained a GPT using cosine similarity regression rather than CEL classification to predict future concept vectors. then what i'd be able to do is after the first t tokens are created this meta-model could use the concept vector that's outputted to create all the future concept vectors, then i could run the actual model in parallel for inference

in a future version i would like to allow the model to carry over all previous concept vectors it has made from this sequence rather than just the most recent one

at some point i'd like to solve that dynamic context length problem. one idea i've got is to use a dataset with actual documents that end in \<endoftext> tokens. basically i'd take the total sequence length of the document and divide by k, then do my batch training on context lengths of that size. i'd have to match up documents of the same size. and i'd have to switch from learned pos embeddings to RoPE. 

Another idea to potentially explore is to use the unused outputs of the regression head for something. maybe hyperparameter control or creating higher-level concept vectors

after that i would like to train the model on a dataset of separate documents where each document provides its own concept vector, so this model can essentially be used to extend my obsidian graph

a big assumption with this model is that the embedding space is actually capable of carrying surprisingly complicated concepts, far more than the number of tokens

another half-baked idea for the generation process is to 

#### !!!! DO NOT RUN THIS FIRST CELL UNLESS YOU HAVE THE SAME VENV PATH ISSUE THAT I DO

In [1]:
import sys
sys.path.append('/Users/tunadorable/local-repos/learning_medusa/venv/lib/python3.11/site-packages')

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import time
import random

#### !!!! ONLY FOR APPLE SILICON
make your own if u use cuda

In [2]:
device = 'mps' if torch.backends.mps.is_available() else 'cpu'

In [3]:
# hyperparameters
b = 3 # how many independent sequences will we process in parallel?
t = 8 # what is the maximum context length for predictions?
max_iters = 100
eval_interval = 10
lr = 3e-4 # learning rate for each backprop step
eval_iters = 20
d = 16 # embedding aka hidden dimension
h = 4 # number of attention heads
l = 4 # number of transormer layers
dropout = 0.2 # % of parameters to ignore every iteration
l2 = 0.01 # multiplier for our L2 norm to encourage sparsity

k = 4

In [4]:
# the dataset is TinyShakespeare
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [5]:
# here are all the unique characters that occur in this text
# we'll be using individual characters instead of tokens
chars = sorted(list(set(text)))
chars.append('') # a learnable token we'll maybe use later
v = len(chars)
print(chars, v)

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ''] 66


In [6]:
# create a mapping from characters to integers & vice versa
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [7]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]
print(len(train_data), len(val_data))

1003854 111540


### need to make the model capable of using this weird batch setup later

In [8]:
def get_batch(split, k=k, b=b, t=t):
    # Assume train_data and val_data are defined outside this function
    data = train_data if split == 'train' else val_data
    max_index = len(data) - k * t
    ix = torch.randint(max_index, (b,))

    # Initialize x and y
    x = torch.zeros((k, b, t), dtype=data.dtype).to(device)
    y = torch.zeros((k, b, t), dtype=data.dtype).to(device)

    # Fill in x and y
    for i in range(k):
        x[i] = torch.stack([data[j+i*t:j+i*t+t] for j in ix])
        y[i] = torch.stack([data[j+i*t+1:j+i*t+t+1] for j in ix])

    return x, y

In [9]:
# so you can see what the tokenized data looks like
x,y = get_batch('train')
print("x ", x.shape, "\n", x)
print("y ", y.shape, "\n", y)
t1, t2, t3, t4 = decode(x[0,0].tolist()), decode(x[1,0].tolist()), decode(x[2,0].tolist()), decode(x[3,0].tolist())
print("the spaces are messed up only bc of how print() works: \n", t1,t2,t3,t4)

x  torch.Size([4, 3, 8]) 
 tensor([[[43, 39, 56, 58, 46,  1, 58, 53],
         [50,  1, 63, 53, 59, 56,  1, 39],
         [ 1, 44, 56, 53, 51,  1, 46, 43]],

        [[ 1, 49, 52, 53, 61,  8,  0,  0],
         [41, 58, 57,  1, 39, 56, 43,  1],
         [52, 41, 43,  1, 47, 57,  1, 53]],

        [[14, 30, 33, 32, 33, 31, 10,  0],
         [55, 59, 43, 43, 52, 57,  8,  0],
         [44,  1, 57, 53,  1, 55, 59, 47]],

        [[28, 56, 39, 63,  6,  1, 50, 43],
         [ 0, 28, 17, 30, 16, 21, 32, 13],
         [41, 49,  1, 41, 53, 52, 42, 47]]])
y  torch.Size([4, 3, 8]) 
 tensor([[[39, 56, 58, 46,  1, 58, 53,  1],
         [ 1, 63, 53, 59, 56,  1, 39, 41],
         [44, 56, 53, 51,  1, 46, 43, 52]],

        [[49, 52, 53, 61,  8,  0,  0, 14],
         [58, 57,  1, 39, 56, 43,  1, 55],
         [41, 43,  1, 47, 57,  1, 53, 44]],

        [[30, 33, 32, 33, 31, 10,  0, 28],
         [59, 43, 43, 52, 57,  8,  0,  0],
         [ 1, 57, 53,  1, 55, 59, 47, 41]],

        [[56, 39, 63,  6,  1,

### for now since we don't have generate() yet we'll use the regular one

In [10]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - t, (b,))
    x = torch.stack([data[i:i+t] for i in ix])
    y = torch.stack([data[i+1:i+t+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [11]:
# so you can see what the tokenized data looks like
x,y = get_batch('train')
print("x ", x.shape, "\n", x)
print("y ", y.shape, "\n", y)

x  torch.Size([3, 8]) 
 tensor([[46, 43,  1, 46, 47, 45, 46,  5],
        [47, 52, 60, 43, 41, 58, 47, 60],
        [ 1, 58, 46, 43, 51, 11,  1, 39]])
y  torch.Size([3, 8]) 
 tensor([[43,  1, 46, 47, 45, 46,  5, 57],
        [52, 60, 43, 41, 58, 47, 60, 43],
        [58, 46, 43, 51, 11,  1, 39, 52]])


In [12]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval() # sets model to eval mode
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, c_vecs, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train() # just resets to training mode
    return out

In [13]:
class FeedForward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, d):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d, 4 * d), # the 4 is arbitrary, but i wouldn't go smaller
            nn.ReLU(), 
            nn.Linear(4 * d, d),
            nn.Dropout(dropout))

    def forward(self, x):
        return self.net(x)

In [14]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(d, head_size, bias=False)
        self.query = nn.Linear(d, head_size, bias=False)
        self.value = nn.Linear(d, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(1+t,1+t))) # mask future timestesps # 1+ for the prepended concept vec
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        b,t,d = x.shape
        k = self.key(x)   # (b,t,d/h)
        q = self.query(x) # (b,t,d/h)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (b, t, d/h) @ (b, d/h, t) -> (b, t, t)
        wei = wei.masked_fill(self.tril[:t, :t] == 0, float('-inf')) # (b, t, t)
        wei = F.softmax(wei, dim=-1) # (b, t, t)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (b,t,d/h)
        out = wei @ v # (b, t, t) @ (b, t, d/h) -> (b, t, d/h)
        return out

In [15]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, h, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(h)])
        self.proj = nn.Linear(head_size * h, d)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([head(x) for head in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [16]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, d, h):
        # d: embedding dimension, h: the number of heads we'd like
        super().__init__()
        head_size = d // h # the double backslash just makes the output an int instead of float
        self.sa = MultiHeadAttention(h, head_size)
        self.ffwd = FeedForward(d)
        self.ln = nn.LayerNorm(d, elementwise_affine=False)

    def forward(self, x):
        x = x + self.sa(self.ln(x))
        x = x + self.ffwd(self.ln(x))
        return x

In [22]:
class conceptGPTv9(nn.Module):
    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(v, d)
        self.vocab_len = v
        
        # simple learned positional encodings rather than sine or RoPE
        self.position_embedding_table = nn.Embedding(t+1, d) # +1 for c or the learnable token
        
        # bulk of the beast
        self.blocks = nn.Sequential(*[Block(d, h) for _ in range(l)]) 
        
        # output head
        self.lm_head = nn.Linear(d, v) 
        
        # apparently layernorm by default actually adds a linear layer & bias unless you specificaly specify false
        self.ln = nn.LayerNorm(d, elementwise_affine=False) # final layer norm
        
        # the concept head
        self.conc_head = FeedForward(d)

        # Initialize Cosine Similarity module here
        self.cosine_similarity = nn.CosineSimilarity(dim=1)

        # according to Andrej Karpathy this _init_weights method is better than default
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
    
    def forward(self, idx=None, targets=None, c_vecs=None, c_hat=None, verbose=False):
        # ik it's crazy but you can in fact not pass in a sequence & still have it forward pass
        b, t = idx.shape if idx else 1,0
            
        # if there's no input concept vector we use the learned token ''
        if c_vecs is None:
            # v-1 is the index of the '' token we added
            c_ind = (self.vocab_len-1)*torch.ones((b,1),device=device,dtype=torch.long) # (b,1)

            # turn it into the vector parts for residual later
            c_vecs = self.token_embedding_table(c_ind) # (b,1,d)

        # regular minimal GPT stuff but with 1+t as our context length
        pos_emb = self.position_embedding_table(torch.arange(1+t, device=device)) # (1+t,d)
        if idx:
            tok_emb = self.token_embedding_table(idx) # (b,t,d)
            x = self.ln(torch.cat((c_vecs,tok_emb),dim=1) + pos_emb) # (b,1+t,d)
        else:
            x = self.ln(c_vecs + pos_emb)
        
        # the bulk of the beast
        x = self.ln(self.blocks(x)) # (b,1+t,d) -> (b,1+t,d)
        
        # the regular next-token prediction head
        logits = self.lm_head(x)[:,1:,:] # ((b,1+t,d)@(d,v))[:,1:,:] -> (b,t,v)

        # the concept head is just a 2-layer feedforward, a splice to make it 1 vector per sequence, then a layernorm
        c_out = self.ln(self.conc_head(x)[:,0,:]) # (b,1_t,d) -> (b,d)
        # the other indicies could be used for something else in the future

        if targets is None:
            # If we're not training at all, we can ignore loss
            loss = None
        else:
            # Regular NTP loss
            b, t, v = logits.shape
            ntp_loss = F.cross_entropy(logits.reshape(b*t, v), targets.reshape(b*t))
            
            if c_hat:
                # Cosine similarity loss for concept vectors
                # this is similar to doing a regression since all layernormed vectors have roughly the same radius anyways
                # except unlike MSE or MAE, it lets us interpret all c's as vectors in embedding space
                similarity = self.cosine_similarity(c_out, c_hat)

                # Maximizing cosine similarity is equivalent to minimizing 1 - cosine similarity
                c_loss = 1 - similarity.mean()  
        
                # This will likely need a parameter to balance the two
                loss = ntp_loss + c_loss
            else:
                # If we're on the first run of training
                # aka don't have a c_hat
                # aka we're at the end of the sequence
                loss = ntp_loss
        
        return logits, c_out, loss

# Training

if you don't want to do your own training just scroll down

In [23]:
model = conceptGPT().to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in model.parameters())/1e3, 'K parameters')

17.122 K parameters


In [24]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=l2)

In [25]:
start_time = time.time()
for iter in range(max_iters):

    # sample a batch of data
    xb, yb = get_batch('train')
    
    # train
    logits, c_vecs, loss = model(xb, targets=yb)
    
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        current_time = time.time()
        elapsed_time = current_time - start_time
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}, time elapsed: {elapsed_time:.2f} seconds")

step 0: train loss 4.1938, val loss 4.1904, time elapsed: 0.03 seconds
step 10: train loss 4.1714, val loss 4.1724, time elapsed: 0.37 seconds
step 20: train loss 4.1538, val loss 4.1524, time elapsed: 0.68 seconds
step 30: train loss 4.1403, val loss 4.1292, time elapsed: 0.99 seconds
step 40: train loss 4.1088, val loss 4.1106, time elapsed: 1.30 seconds
step 50: train loss 4.0828, val loss 4.0995, time elapsed: 1.67 seconds
step 60: train loss 4.0389, val loss 4.0380, time elapsed: 2.04 seconds
step 70: train loss 4.0155, val loss 4.0089, time elapsed: 2.37 seconds
step 80: train loss 3.9557, val loss 3.9642, time elapsed: 2.67 seconds
step 90: train loss 3.9307, val loss 3.9327, time elapsed: 2.99 seconds
step 99: train loss 3.8744, val loss 3.8953, time elapsed: 3.29 seconds


In [21]:
import math

model.eval()

input_str = "JULIET:\nO Romeo, Romeo! wherefore art thou R" # the classic line
idx = torch.tensor([encode(input_str)], dtype=torch.long, device=device)
print("idx: ", idx.shape, idx)
c_inp=None
max_new_tokens=25
temperature=1.0

# idx is (b, i) array of indices in the current context
b, i = idx.shape
#assert temperature >= 0

#### getting the first concept vector(s)
# the subsequence we'll be performing inference on
inp = idx[:,0:t] #"JULIET:\n"
# we want that first outputted concept vector
logits, c_vecs, loss = model(idx=inp)

# getting concept vectors for the next few length t parts of the input context
context_len = len(idx)
context_chunks = context_len // t
for j in range(1,context_chunks): # "O Romeo," -> " Romeo! " -> "wherefor" -> "e art th" 
    # the subsequence we'll be performing inference on
    inp = idx[:,j:j+t]

    # all we care about is that next concept vector
    logits, c_vecs, loss = model(idx=inp, c_vecs=c_vecs)

# defining c_inp since we won't want to use the newest c_vecs every time
c_inp = c_vecs 

# beginning generation wherever the context leaves off
inp = idx[:,context_chunks*t:]
partial_final_context_chunk = len(idx) % t
for j in range(partial_final_context_chunk, t - partial_final_context_chunk): # "ou R"+
    # now we're looking for that next token
    logits, c_vecs, loss = model(idx=inp, c_vecs=c_vecs)

    # focus only on the last time step
    logits = logits[:, -1, :] # becomes (b, d)

    # scale logits by the temperature
    logits = logits / (temperature+1e-10)

    # apply softmax to get probabilities
    probs = F.softmax(logits, dim=-1) # (b, d)

    # sample from the distribution
    idx_next = torch.multinomial(probs, num_samples=1) # (b, 1)

    # to be inputted next inference run
    inp = torch.concat((inp, idx_next),dim=1)

    # keeping track of our total sequence
    idx = torch.concat((idx, idx_next),dim=1)

# and then after we've finished filling in that partially-filled chunk we want that final concept vector
c_inp = c_vecs

# all of the remaining full chunks as defined by max_new_tokens
full_chunks = (context_len +m ax_new_tokens) // t
for k in range(context_len + 1, full_chunks):
    inp = None
    for j in range(t):
        logits, c_vecs, loss = model(idx=inp, c_vecs=c_inp

# the final remainder as defined by the number of chunks & max_new_tokens
final_remainder = (context_len + max_new_tokens) % t
for j in range(final_remainder):
    












    
# the bulk of the iterations where we use the context's context vectors to make subsequences length t of tokens
next_tokens = max_new_tokens - (t - (len(idx) % t))
main_iterations = next_tokens // t
for j in range(main_iterations):


# the tail end remainder portion that's not a full length t subsequence
remainder_iterations = next_tokens % t
for j in range(remainder_iterations):
    

# should this go here or in the loop?
#if c_inp == None:
    # if there's no input concept vector we use the learned token ''
    #c_inp = (v-1)*torch.ones((b,1),device=device,dtype=torch.long)

# crop idx to maximum context length then concat w/ c_inp
#inp = torch.cat((c_inp, idx[:, -t:]), dim=1) # (b,1)&(b,t)-> (b,1+t)
# actually forward() does that for us so we can just not pass anything real in
inp = idx[:,-t:]
print("inp: ", inp.shape, inp)



for j in range(t):
    print("j: ", j)

    # get the predictions
    logits, c_vecs, loss = model(inp) # (b,t,d), float, (b,d)

    # focus only on the last time step
    logits = logits[:, -1, :] # becomes (b, d)

    # scale logits by the temperature
    logits = logits / (temperature+1e-10)

    # apply softmax to get probabilities
    probs = F.softmax(logits, dim=-1) # (b, d)

    # sample from the distribution
    idx_next = torch.multinomial(probs, num_samples=1) # (b, 1)

for j in range(math.ceil(max_new_tokens / t)):
    for k in range(t):
        print("j: ", j, " k: ", k)

        # later i need to implement this but with functions bc right now this code breaks out of both loops not just the inner
        #if j == 0 & k == 0: 
            #continue
        
        

        # append to master idx
        idx = torch.cat((idx, idx_next), dim=1)

        # remove concept vec & append sampled index to the running sequence
        idx_temp = torch.cat((inp[:,2:], idx_next), dim=1) # (b, t+1)

        # crop idx to maximum context length then concat w/ c_inp
        inp = torch.cat((c_inp, idx_temp[:, -t:]), dim=1) # (b,1)&(b,t)-> (b,1+t)
        
        # get the predictions
        logits, c_vecs, loss = model(inp, c_inp=c_inp) # (b,t,d), float, (b,d)

    print("c_vecs: ", c_vecs.shape, c_vecs)
    c_inp = c_vecs

SyntaxError: invalid syntax (1851202348.py, line 29)

In [ ]:
print(decode(idx[0].tolist()))

## save the trained model

In [22]:
torch.save(model.state_dict(), f'models/{model.__class__.__name__}_b{b}_t{t}_d{d}_h{h}_l{l}_lr{lr}_drop{dropout}_l2-{l2}_m{m}_mdiscount{medusa_discount:.2f}_{time.strftime("%Y-%m-%d|%H-%M-%S")}.pth')

# Load a saved model

In [21]:
model = medusaGPT().to(device)  # Initialize a model with the same architecture

# Load the saved state dictionary
model.load_state_dict(torch.load('models/medusa_b24_t128_d128_h8_l8_lr0.0003_drop0.2_l2-0.01_m5_mdiscount0.80_2024-01-25|23-31-12.pth'))
# that's the better model of the two that I trained. The extra heads were useless tho

# If you plan to continue training the model, switch to training mode
#model.train()

# If you only plan to do inference, switch to evaluation mode
model.eval()

medusaGPT(
  (token_embedding_table): Embedding(65, 128)
  (position_embedding_table): Embedding(128, 128)
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-7): 8 x Head(
            (key): Linear(in_features=128, out_features=16, bias=False)
            (query): Linear(in_features=128, out_features=16, bias=False)
            (value): Linear(in_features=128, out_features=16, bias=False)
            (dropout): Dropout(p=0.2, inplace=False)
          )
        )
        (proj): Linear(in_features=128, out_features=128, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
      )
      (ffwd): FeedFoward(
        (net): Sequential(
          (0): Linear(in_features=128, out_features=512, bias=True)
          (1): ReLU()
          (2): Linear(in_features=512, out_features=128, bias=True)
          (3): Dropout(p=0.2, inplace=False)
        )
      )
      (ln1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
   

## Regular GPT Inference

So it seems to me like regular GPT inference, although not the fastest, gives the best output quality. You'll see why my versions have worse output quality later

In [22]:
%%time # to keep track of how long it takes
input_str = "JULIET:\nO Romeo, Romeo! wherefore art thou R" # the classic line
context_tensor = torch.tensor([encode(input_str)], dtype=torch.long, device=device)
output = model.generate_gpt(context_tensor, max_new_tokens=218)
output_str = decode(output[0].tolist())
print(output_str)

JULIET:
O Romeo, Romeo! wherefore art thou Richi? lips, Citizen?

ARCHTGGS:
Here's here: I'll forth his son in those fear;
Lord and far his sight, and not be kneel,
Which shad my many brotherful sea,
And a word a ben the people.

MENENIUS:
Upon my name, sir: I h
CPU times: user 32.7 s, sys: 1.71 s, total: 34.4 s
Wall time: 34.1 s


# Medusa's first sister Stheno (the aggressive one)

ChatGPT said:
Her name translates to "strength" or "forceful". Stheno was the eldest and most fierce of the sisters, known for her strength and ferocity. 

So I guess since this generation is strictly doing greedy decoding i'll name it after her

The idea with Stheno is we're just imementing a tiny foundational concept of Medusa. Basically, while next-token prediction does prediction in parallel not only across batches but also (more importantly here and less frequently talked about) across time-steps in a sequence like this:
<table border="1">
    <thead>
        <tr>
            <th>input (<code>prompt</code>)</th>
            <td>'R'</td>
            <td>'o'</td>
            <td>'m'</td>
            <td>'e'</td>
            <td>'o'</td>
        </tr>
    </thead>
    <tbody>
        <tr>
            <th>NTP output</th>
            <td>'o'</td>
            <td>'m'</td>
            <td>'e'</td>
            <td>'o'</td>
            <td>'?'</td>
        </tr>
    </tbody>
</table>

Our medusa heads will make output predictions multiple steps into the future that look like this
<table border="1">
    <thead>
        <tr>
            <th>input (<code>prompt</code>)</th>
            <td>'R'</td>
            <td>'o'</td>
            <td>'m'</td>
            <td>'e'</td>
            <td>'o'</td>
        </tr>
    </thead>
    <tbody>
        <tr>
            <th>medusa head 1</th>
            <td>'m'</td>
            <td>'e'</td>
            <td>'o'</td>
            <td>'?'</td>
            <td>'\n'</td>
        </tr>
    </tbody>
</table>
You can imagine what it looks like for further medusa heads.

The question now is how can we take advantage of this to speed up inference? Basically we take the input text and the output from each attention head and run those into the model, so the steps assuming two medusa heads are
1) input prompt 'R' and recieve output
<table border="1">
    <thead>
        <tr>
            <th>input</th>
            <td>'R'</td>
            <td></td>
            <td></td>
        </tr>
    </thead>
    <tbody>
        <tr>
            <th>NTP output</th>
            <td>'o'</td>
            <td></td>
            <td></td>
        </tr>
        <tb>
            <th>medusa head 1</th>
            <td></td>
            <td>'m'</td>
            <td></td>
        </tb>
    </tbody>
    <tbody>
        <tb>
            <th>medusa head 2</th>
            <td></td>
            <td></td>
            <td>'e'</td>
        </tb>
    </tbody>
</table>
2) input sequence 'Rome' and recieve output
<table border="1">
    <thead>
        <tr>
            <th>input</th>
            <td>'R'</td>
            <td>'o'</td>
            <td>'m'</td>
            <td>'e'</td>
            <td></td>
            <td></td>
        </tr>
    </thead>
    <tbody>
        <tr>
            <th>NTP output</th>
            <td>'o'</td>
            <td>'m'</td>
            <td>'e'</td>
            <td>'o'</td>
            <td></td>
            <td></td>
        </tr>
        <tb>
            <th>medusa head 1</th>
            <td></td>
            <td></td>
            <td></td>
            <td></td>
            <td>'?'</td>
            <td></td>
        </tb>
    </tbody>
    <tbody>
        <tb>
            <th>medusa head 2</th>
            <td></td>
            <td></td>
            <td></td>
            <td></td>
            <td></td>
            <td>'\n'</td>
        </tb>
    </tbody>
</table>

3) Notice how our NTP output has now confirmed for us that medusa heads 1 and 2 were correct because the NTP row matches up with the input. Now, even though we've only run the model twice, we've obtained 4 tokens 'omeo' that are confirmed to be predicted by the regular NTP algorithm.


However, what if the medusa heads hadn't been right? Let's rewind
1) input prompt 'R' and recieve output
<table border="1">
    <thead>
        <tr>
            <th>input</th>
            <td>'R'</td>
            <td></td>
            <td></td>
        </tr>
    </thead>
    <tbody>
        <tr>
            <th>NTP output</th>
            <td>'o'</td>
            <td></td>
            <td></td>
        </tr>
        <tb>
            <th>medusa head 1</th>
            <td></td>
            <td>'m'</td>
            <td></td>
        </tb>
    </tbody>
    <tbody>
        <tb>
            <th>medusa head 2</th>
            <td></td>
            <td></td>
            <td>'u'</td>
        </tb>
    </tbody>
</table>
3) Maybe medusa head 2 thinks we're going to predict the name 'Romulus'? Let's input this string 'Romu' and see what we get
<table border="1">
    <thead>
        <tr>
            <th>input</th>
            <td>'R'</td>
            <td>'o'</td>
            <td>'m'</td>
            <td>'u'</td>
            <td></td>
            <td></td>
        </tr>
    </thead>
    <tbody>
        <tr>
            <th>NTP output</th>
            <td>'o'</td>
            <td>'m'</td>
            <td>'e'</td>
            <td>'l'</td>
            <td></td>
            <td></td>
        </tr>
        <tb>
            <th>medusa head 1</th>
            <td></td>
            <td></td>
            <td></td>
            <td></td>
            <td>'u'</td>
            <td></td>
        </tb>
    </tbody>
    <tbody>
        <tb>
            <th>medusa head 2</th>
            <td></td>
            <td></td>
            <td></td>
            <td></td>
            <td></td>
            <td>'s'</td>
        </tb>
    </tbody>
</table>
3) Notice how the NTP output gives us an 'e' instead of the expected 'u'. It's unfortunate that medusa head 2's prediction was wrong, but medusa head 1's prediction was correct so we're still correct and this "checking" step has still given us one output 'e' meaning we're still ahead of regular next-token prediction. So far we've run the model twice but received 3 tokens, overall still putting us ahead. For the next step we'll just input the confirmed tokens we've recieved so far, which are 'Rome'

A key insight to this process is that we're considering NTP's output to be the "ground truth" during inference, and we get to use it as confirmation because of how transformers are trained in parallel across steps in the sequence. Thanks to this, even when every single medusa head is wrong we still get 1 token out of the NTP head. If our medusa heads are trained extremely poorly then this process should reduce down to NTP plus the marginal extra compute from the heads & checking, but if the heads are trained well then we can get a pretty significant speed increase

Now let's give it a try implementing this algorithm which I'll call Stheno, the ugliest sister

##### NOTE: I frequently refer to 't' throughout the remainder of the code. keep in mind t does not remain static, it grows over time. If it gets confusing just ignore any references to t and know that it could mean t give or take 1+m

In [4]:
def generate_Stheno(model, idx, max_runs):
    # Ensure idx is a single sequence. This won't work for batched inference
    assert idx.size(0) == 1, "idx must be of size (1, t)"
    print("idx: ", idx.shape)
    
    # crop idx to the last block_size tokens jic it's too long
    input_cond = idx[:, -t:] 
    print("idx_cond should be size (1, t): ", input_cond.shape)
          
    # get the predictions
    logits, loss, mlogits = model(input_cond) # (1,t,v), int, and (m,1,t,v)
    print("logits should be (1,t,v): ", logits.shape)
    print("mlogits should be (m,1,t,v): ", mlogits.shape)
    
    # we only want the medusa preditions for the newest future timestep
    mlogits = mlogits[...,-1,:] # becomes (m,1,v)
    mlogits = mlogits.squeeze(dim=1) # (m,v)
    print("mlogits should be (m,v): ", mlogits.shape)
    
    # actual medusa does a whole proabability attention mask tree thing, 
    #   but here we'll assume greedy decoding on snake heads
    idx_m_prev = torch.argmax(mlogits, dim=-1, keepdim=True).t() # (1,m)
    print("idx_m_prev should be (1,m): ", idx_m_prev.shape)
    # we name it _prev since it'll be used on the next loop
    
    # medusa requires greedy decoding for the first regular t+1'th token
    idx_ntp = torch.argmax(logits, dim=-1, keepdim=True).squeeze(dim=2) # (1, t)
    print("idx_ntp should be (1, t): ", idx_ntp.shape)
    
    # append sampled index to the running sequence
    idx = torch.cat((idx, idx_ntp[:,-1].unsqueeze(dim=0)), dim=1) # (1,t)+(1,1) -> (1, t+1)
    print("idx should be (1, t+1): ", idx.shape)

    # keep track of how many tokens we get per model inference
    tok_per_inf = [1]
    
    for _ in range(max_runs-1): # -1 since one iteration was done above
        ###### I think this is where i begin the looped part. above stuff should be before the for loop
        
        # so now we have idx shape (1,t+1) of context tokens plus 1 predicted token
        # and idx_m_prev shape  (1,m) of speculative tokens to check against later & include some
        # want to know whether model actually would've predicted the tokens in idx_m so we need to run it with them & compare
        
        # first we construct a tensor composed of the initial context, the single ntp token, and then the speculative tokens
        inp = torch.cat((idx, idx_m_prev), dim=1) # (1,t+1) & (1,m) -> (1,t+1+m)
        print("input should be (1,t+1+m): ", inp.shape)
        
        # but since the model can't take more than t inputs
        # note: t is max content limit, but input might be shorter. if so that's fine
        input_cond = inp[:, -t:] 
        # (1,t+1+m) -> (1,t) where first t-1-m are prior context, the t-m'th is ntp, & last m are candidates
        print("input_cond should be (1,t): ", input_cond.shape)
        
        # then we pass it in
        logits, loss, mlogits = model(input_cond) # (1,t,v), tensor of a single float, & (m,1,t,v)
        
        # we're just greedy decoding so there's no need to softmax
        idx_ntp = torch.argmax(logits, dim=-1, keepdim=True).squeeze(dim=2) # (1,t,v) -> (1, t)
        print("idx_ntp should be (1, t): ", idx_ntp.shape)
        
        # now for making the comparison we need to ignore the indices that were part of the context
        # the regular ntp prediction can also be ignored
        # we just need to know the number of previous turn's medusa preds to keep
        idx_check = idx_ntp[:,-(m+1):-1] # (1,t) -> (1,m)
        print("idx_check should be (1,m): ", idx_check.shape, "\n", idx_check)
        
        # let's start checking whether they match
        match_tensor = (idx_m_prev == idx_check).int()
        #(1,m) of ints & (1,m) of ints -> (1,m) of 1's and 0's
        print("match_tensor should be (1,m) of 1's and 0's: ", match_tensor.shape, "\n", match_tensor)
        
        # then define our "definitely good" indices as those which are 1's for the similarity check
        # We invert the tensor so that all zeros & 1's flip
        # Find the first 1 in each row. The max function returns the index of the first occurrence of the maximum value.
        # We add one extra zero at the end of each row to handle rows that contain no zeros.
        pad = torch.ones(match_tensor.size(0), 1, dtype=match_tensor.dtype, device=device)
        print("pad: ", pad.shape, "\n", pad)
        padded_tensor = torch.cat((1 - match_tensor, pad), dim=1)
        print("padded_tensor: ", padded_tensor.shape, "\n", padded_tensor)
        zero_positions = padded_tensor.argmax(dim=1)
        
        # Adjust indices where the last position is selected (meaning there were no zeros)
        zero_positions[zero_positions >= match_tensor.size(1)] = match_tensor.size(1)
        print("zero_positions: ", zero_positions.shape, "\n", zero_positions)
        
        # Create a range tensor
        range_tensor = torch.arange(match_tensor.size(1), device=device).unsqueeze(0).expand_as(match_tensor)
        print("range_tensor: ", range_tensor.shape, "\n", range_tensor)
        
        # Create a mask where each element is 1 if it is before the first zero in its row
        mask = range_tensor < zero_positions.unsqueeze(1)
        print("mask: ", mask.shape, "\n", mask)
        
        # Apply the mask and sum along each row
        result = (match_tensor * mask).sum(dim=1).item()
        print("result: ", result)

        tok_per_inf.append(result+1)
        
        # so now i've got result which is an integer between 0 and m (inclusive)
        # i need to use it as an index on the actual medusa head outputs of interest and then append that to our sequence
        # but we also get to use the regular ntp prediction right after any of last run's accepted medusa head predictions
        print("idx: ", idx.shape, "\n", idx)
        idx_m_prev = idx_m_prev[:,:result]
        print("idx_m_prev should be (1,result): ", idx_m_prev.shape, "\n", idx_m_prev)
        idx_ntp = idx_ntp[:,-1-m+result].unsqueeze(dim=0)
        print("idx_ntp should be (1,1): ", idx_ntp.shape, "\n", idx_ntp)
        idx = torch.cat((idx, idx_m_prev, idx_ntp),dim=1)
        print("idx should be (1,t+result+1: ", idx.shape, "\n", idx)
        
        # so now we have idx of size (1,t+1+result)
        # and idx_m of size (1,m)
        # wait but there's a problem. If we didn't use the largest result=m then 
        #   idx_m is for items too far into the future. we need to pass forward an idx_m that's relevant
        # ngl i'm too tired to think about it so i'm assuming i can use the same index i used for idx_ntp
        mlogits = mlogits[...,-1-m+result,:].squeeze(dim=1) # (m,1,t,v) -> (m,1,v)
        print("mlogits: ", mlogits.shape)
        idx_m_prev = torch.argmax(mlogits, dim=-1, keepdim=True).t() # (m,1,v) -> (1,m)
        print("idx_m_prev: ", idx_m_prev.shape, "\n", idx_m_prev)
        # we name it _prev since it'll be used on the next loop
        
    return idx, tok_per_inf

In [24]:
%%time
input_str = "JULIET:\nO Romeo, Romeo! wherefore art thou R"
context_tensor = torch.tensor([encode(input_str)], dtype=torch.long, device=device)
output, tok_per_inf = generate_Stheno(model, context_tensor, max_runs=100)
output_str = decode(output[0].tolist())

idx:  torch.Size([1, 44])
idx_cond should be size (1, t):  torch.Size([1, 44])
logits should be (1,t,v):  torch.Size([1, 44, 65])
mlogits should be (m,1,t,v):  torch.Size([5, 1, 44, 65])
mlogits should be (m,v):  torch.Size([5, 65])
idx_m_prev should be (1,m):  torch.Size([1, 5])
idx_ntp should be (1, t):  torch.Size([1, 44])
idx should be (1, t+1):  torch.Size([1, 45])
input should be (1,t+1+m):  torch.Size([1, 50])
input_cond should be (1,t):  torch.Size([1, 50])
idx_ntp should be (1, t):  torch.Size([1, 50])
idx_check should be (1,m):  torch.Size([1, 5]) tensor([[51, 43, 12, 12,  0]], device='mps:0')
match_tensor should be (1,m) of 1's and 0's:  torch.Size([1, 5]) tensor([[1, 1, 0, 1, 1]], device='mps:0', dtype=torch.int32)
pad:  torch.Size([1, 1]) tensor([[1]], device='mps:0', dtype=torch.int32)
padded_tensor:  torch.Size([1, 6]) tensor([[0, 0, 1, 0, 0, 1]], device='mps:0', dtype=torch.int32)
zero_positions:  torch.Size([1]) tensor([2], device='mps:0')
range_tensor:  torch.Size([1,

idx_ntp should be (1, t):  torch.Size([1, 67])
idx_check should be (1,m):  torch.Size([1, 5]) tensor([[58,  1, 47, 47, 44]], device='mps:0')
match_tensor should be (1,m) of 1's and 0's:  torch.Size([1, 5]) tensor([[1, 1, 0, 0, 0]], device='mps:0', dtype=torch.int32)
pad:  torch.Size([1, 1]) tensor([[1]], device='mps:0', dtype=torch.int32)
padded_tensor:  torch.Size([1, 6]) tensor([[0, 0, 1, 1, 1, 1]], device='mps:0', dtype=torch.int32)
zero_positions:  torch.Size([1]) tensor([2], device='mps:0')
range_tensor:  torch.Size([1, 5]) tensor([[0, 1, 2, 3, 4]], device='mps:0')
mask:  torch.Size([1, 5]) tensor([[ True,  True, False, False, False]], device='mps:0')
result:  2
idx:  torch.Size([1, 62]) tensor([[22, 33, 24, 21, 17, 32, 10,  0, 27,  1, 30, 53, 51, 43, 53,  6,  1, 30,
         53, 51, 43, 53,  2,  1, 61, 46, 43, 56, 43, 44, 53, 56, 43,  1, 39, 56,
         58,  1, 58, 46, 53, 59,  1, 30, 53, 51, 43, 12,  0,  0, 28, 56, 53, 60,
         53, 57, 58, 10,  0, 35, 46, 39]], device='mps:

idx should be (1,t+result+1:  torch.Size([1, 77]) tensor([[22, 33, 24, 21, 17, 32, 10,  0, 27,  1, 30, 53, 51, 43, 53,  6,  1, 30,
         53, 51, 43, 53,  2,  1, 61, 46, 43, 56, 43, 44, 53, 56, 43,  1, 39, 56,
         58,  1, 58, 46, 53, 59,  1, 30, 53, 51, 43, 12,  0,  0, 28, 56, 53, 60,
         53, 57, 58, 10,  0, 35, 46, 39, 58,  1, 47, 57,  1, 58, 46, 43,  1, 61,
         53, 56, 50, 42, 12]], device='mps:0')
mlogits:  torch.Size([5, 65])
idx_m_prev:  torch.Size([1, 5]) tensor([[0, 0, 1, 1, 1]], device='mps:0')
input should be (1,t+1+m):  torch.Size([1, 82])
input_cond should be (1,t):  torch.Size([1, 82])
idx_ntp should be (1, t):  torch.Size([1, 82])
idx_check should be (1,m):  torch.Size([1, 5]) tensor([[ 0,  0, 15, 15, 24]], device='mps:0')
match_tensor should be (1,m) of 1's and 0's:  torch.Size([1, 5]) tensor([[1, 1, 0, 0, 0]], device='mps:0', dtype=torch.int32)
pad:  torch.Size([1, 1]) tensor([[1]], device='mps:0', dtype=torch.int32)
padded_tensor:  torch.Size([1, 6]) te

idx should be (1,t+result+1:  torch.Size([1, 91]) tensor([[22, 33, 24, 21, 17, 32, 10,  0, 27,  1, 30, 53, 51, 43, 53,  6,  1, 30,
         53, 51, 43, 53,  2,  1, 61, 46, 43, 56, 43, 44, 53, 56, 43,  1, 39, 56,
         58,  1, 58, 46, 53, 59,  1, 30, 53, 51, 43, 12,  0,  0, 28, 56, 53, 60,
         53, 57, 58, 10,  0, 35, 46, 39, 58,  1, 47, 57,  1, 58, 46, 43,  1, 61,
         53, 56, 50, 42, 12,  0,  0, 15, 13, 25, 21, 24, 24, 27, 10,  0, 21,  1,
         61]], device='mps:0')
mlogits:  torch.Size([5, 65])
idx_m_prev:  torch.Size([1, 5]) tensor([[39, 39, 43,  1,  1]], device='mps:0')
input should be (1,t+1+m):  torch.Size([1, 96])
input_cond should be (1,t):  torch.Size([1, 96])
idx_ntp should be (1, t):  torch.Size([1, 96])
idx_check should be (1,m):  torch.Size([1, 5]) tensor([[47, 57, 56,  1, 58]], device='mps:0')
match_tensor should be (1,m) of 1's and 0's:  torch.Size([1, 5]) tensor([[0, 0, 0, 1, 0]], device='mps:0', dtype=torch.int32)
pad:  torch.Size([1, 1]) tensor([[1]], de

idx_ntp should be (1, t):  torch.Size([1, 106])
idx_check should be (1,m):  torch.Size([1, 5]) tensor([[ 1, 58, 46,  1, 58]], device='mps:0')
match_tensor should be (1,m) of 1's and 0's:  torch.Size([1, 5]) tensor([[1, 1, 0, 1, 0]], device='mps:0', dtype=torch.int32)
pad:  torch.Size([1, 1]) tensor([[1]], device='mps:0', dtype=torch.int32)
padded_tensor:  torch.Size([1, 6]) tensor([[0, 0, 1, 0, 1, 1]], device='mps:0', dtype=torch.int32)
zero_positions:  torch.Size([1]) tensor([2], device='mps:0')
range_tensor:  torch.Size([1, 5]) tensor([[0, 1, 2, 3, 4]], device='mps:0')
mask:  torch.Size([1, 5]) tensor([[ True,  True, False, False, False]], device='mps:0')
result:  2
idx:  torch.Size([1, 101]) tensor([[22, 33, 24, 21, 17, 32, 10,  0, 27,  1, 30, 53, 51, 43, 53,  6,  1, 30,
         53, 51, 43, 53,  2,  1, 61, 46, 43, 56, 43, 44, 53, 56, 43,  1, 39, 56,
         58,  1, 58, 46, 53, 59,  1, 30, 53, 51, 43, 12,  0,  0, 28, 56, 53, 60,
         53, 57, 58, 10,  0, 35, 46, 39, 58,  1, 47, 

...

idx should be (1,t+result+1:  torch.Size([1, 237]) tensor([[22, 33, 24, 21, 17, 32, 10,  0, 27,  1, 30, 53, 51, 43, 53,  6,  1, 30,
         53, 51, 43, 53,  2,  1, 61, 46, 43, 56, 43, 44, 53, 56, 43,  1, 39, 56,
         58,  1, 58, 46, 53, 59,  1, 30, 53, 51, 43, 12,  0,  0, 28, 56, 53, 60,
         53, 57, 58, 10,  0, 35, 46, 39, 58,  1, 47, 57,  1, 58, 46, 43,  1, 61,
         53, 56, 50, 42, 12,  0,  0, 15, 13, 25, 21, 24, 24, 27, 10,  0, 21,  1,
         61, 47, 50, 50,  1, 52, 53, 58,  1, 40, 43,  1, 58, 46, 43,  1, 51, 39,
         52, 12,  0,  0, 24, 17, 27, 26, 32, 17, 31, 10,  0, 21,  1, 61, 47, 50,
         50,  1, 52, 53, 58,  1, 57, 43, 43,  1, 58, 46, 43,  1, 51, 39, 52,  1,
         53, 44,  1, 58, 46, 43,  1, 51, 39, 52,  1, 58, 46, 39, 58,  1, 58, 46,
         43, 63,  1, 57, 46, 39, 50, 50,  1, 40, 43,  1, 57, 53,  8,  0,  0, 24,
         33, 15, 21, 27, 10,  0, 21,  1, 61, 47, 50, 50,  1, 52, 53, 58,  1, 57,
         53,  1, 57, 53,  1, 51, 53, 56, 43,  1, 58, 46, 3

idx should be (1,t+result+1:  torch.Size([1, 240]) tensor([[22, 33, 24, 21, 17, 32, 10,  0, 27,  1, 30, 53, 51, 43, 53,  6,  1, 30,
         53, 51, 43, 53,  2,  1, 61, 46, 43, 56, 43, 44, 53, 56, 43,  1, 39, 56,
         58,  1, 58, 46, 53, 59,  1, 30, 53, 51, 43, 12,  0,  0, 28, 56, 53, 60,
         53, 57, 58, 10,  0, 35, 46, 39, 58,  1, 47, 57,  1, 58, 46, 43,  1, 61,
         53, 56, 50, 42, 12,  0,  0, 15, 13, 25, 21, 24, 24, 27, 10,  0, 21,  1,
         61, 47, 50, 50,  1, 52, 53, 58,  1, 40, 43,  1, 58, 46, 43,  1, 51, 39,
         52, 12,  0,  0, 24, 17, 27, 26, 32, 17, 31, 10,  0, 21,  1, 61, 47, 50,
         50,  1, 52, 53, 58,  1, 57, 43, 43,  1, 58, 46, 43,  1, 51, 39, 52,  1,
         53, 44,  1, 58, 46, 43,  1, 51, 39, 52,  1, 58, 46, 39, 58,  1, 58, 46,
         43, 63,  1, 57, 46, 39, 50, 50,  1, 40, 43,  1, 57, 53,  8,  0,  0, 24,
         33, 15, 21, 27, 10,  0, 21,  1, 61, 47, 50, 50,  1, 52, 53, 58,  1, 57,
         53,  1, 57, 53,  1, 51, 53, 56, 43,  1, 58, 46, 3

idx should be (1,t+result+1:  torch.Size([1, 249]) tensor([[22, 33, 24, 21, 17, 32, 10,  0, 27,  1, 30, 53, 51, 43, 53,  6,  1, 30,
         53, 51, 43, 53,  2,  1, 61, 46, 43, 56, 43, 44, 53, 56, 43,  1, 39, 56,
         58,  1, 58, 46, 53, 59,  1, 30, 53, 51, 43, 12,  0,  0, 28, 56, 53, 60,
         53, 57, 58, 10,  0, 35, 46, 39, 58,  1, 47, 57,  1, 58, 46, 43,  1, 61,
         53, 56, 50, 42, 12,  0,  0, 15, 13, 25, 21, 24, 24, 27, 10,  0, 21,  1,
         61, 47, 50, 50,  1, 52, 53, 58,  1, 40, 43,  1, 58, 46, 43,  1, 51, 39,
         52, 12,  0,  0, 24, 17, 27, 26, 32, 17, 31, 10,  0, 21,  1, 61, 47, 50,
         50,  1, 52, 53, 58,  1, 57, 43, 43,  1, 58, 46, 43,  1, 51, 39, 52,  1,
         53, 44,  1, 58, 46, 43,  1, 51, 39, 52,  1, 58, 46, 39, 58,  1, 58, 46,
         43, 63,  1, 57, 46, 39, 50, 50,  1, 40, 43,  1, 57, 53,  8,  0,  0, 24,
         33, 15, 21, 27, 10,  0, 21,  1, 61, 47, 50, 50,  1, 52, 53, 58,  1, 57,
         53,  1, 57, 53,  1, 51, 53, 56, 43,  1, 58, 46, 3

idx should be (1,t+result+1:  torch.Size([1, 256]) tensor([[22, 33, 24, 21, 17, 32, 10,  0, 27,  1, 30, 53, 51, 43, 53,  6,  1, 30,
         53, 51, 43, 53,  2,  1, 61, 46, 43, 56, 43, 44, 53, 56, 43,  1, 39, 56,
         58,  1, 58, 46, 53, 59,  1, 30, 53, 51, 43, 12,  0,  0, 28, 56, 53, 60,
         53, 57, 58, 10,  0, 35, 46, 39, 58,  1, 47, 57,  1, 58, 46, 43,  1, 61,
         53, 56, 50, 42, 12,  0,  0, 15, 13, 25, 21, 24, 24, 27, 10,  0, 21,  1,
         61, 47, 50, 50,  1, 52, 53, 58,  1, 40, 43,  1, 58, 46, 43,  1, 51, 39,
         52, 12,  0,  0, 24, 17, 27, 26, 32, 17, 31, 10,  0, 21,  1, 61, 47, 50,
         50,  1, 52, 53, 58,  1, 57, 43, 43,  1, 58, 46, 43,  1, 51, 39, 52,  1,
         53, 44,  1, 58, 46, 43,  1, 51, 39, 52,  1, 58, 46, 39, 58,  1, 58, 46,
         43, 63,  1, 57, 46, 39, 50, 50,  1, 40, 43,  1, 57, 53,  8,  0,  0, 24,
         33, 15, 21, 27, 10,  0, 21,  1, 61, 47, 50, 50,  1, 52, 53, 58,  1, 57,
         53,  1, 57, 53,  1, 51, 53, 56, 43,  1, 58, 46, 3

idx should be (1,t+result+1:  torch.Size([1, 259]) tensor([[22, 33, 24, 21, 17, 32, 10,  0, 27,  1, 30, 53, 51, 43, 53,  6,  1, 30,
         53, 51, 43, 53,  2,  1, 61, 46, 43, 56, 43, 44, 53, 56, 43,  1, 39, 56,
         58,  1, 58, 46, 53, 59,  1, 30, 53, 51, 43, 12,  0,  0, 28, 56, 53, 60,
         53, 57, 58, 10,  0, 35, 46, 39, 58,  1, 47, 57,  1, 58, 46, 43,  1, 61,
         53, 56, 50, 42, 12,  0,  0, 15, 13, 25, 21, 24, 24, 27, 10,  0, 21,  1,
         61, 47, 50, 50,  1, 52, 53, 58,  1, 40, 43,  1, 58, 46, 43,  1, 51, 39,
         52, 12,  0,  0, 24, 17, 27, 26, 32, 17, 31, 10,  0, 21,  1, 61, 47, 50,
         50,  1, 52, 53, 58,  1, 57, 43, 43,  1, 58, 46, 43,  1, 51, 39, 52,  1,
         53, 44,  1, 58, 46, 43,  1, 51, 39, 52,  1, 58, 46, 39, 58,  1, 58, 46,
         43, 63,  1, 57, 46, 39, 50, 50,  1, 40, 43,  1, 57, 53,  8,  0,  0, 24,
         33, 15, 21, 27, 10,  0, 21,  1, 61, 47, 50, 50,  1, 52, 53, 58,  1, 57,
         53,  1, 57, 53,  1, 51, 53, 56, 43,  1, 58, 46, 3

In [25]:
print("tokens per inference: ", sum(tok_per_inf)/len(tok_per_inf))
print(output_str)

tokens per inference:  2.16
JULIET:
O Romeo, Romeo! wherefore art thou Rome?

Provost:
What is the world?

CAMILLO:
I will not be the man?

LEONTES:
I will not see the man of the man that they shall be so.

LUCIO:
I will not so so more than than the strength of the strength,
And the stre


#### notice that Stheno is restricted to greedy decoding which means the output quality is lower. If you compare the two passages you'll see that the NTP one has far less of a problem with repetition

---

# Medusa's second sister Euryale (the explorative one)


ChatGPT said

Her name means "far-roaming" in Greek. Euryale was known for her loud crying or bellowing. If your architecture is meant to explore a wide range of possibilities or to "roam" extensively through a dataset, the name Euryale might be suitable. Additionally, if your architecture involves a broad or far-reaching search strategy or is notable for 'broadcasting' its findings extensively (analogous to loud crying), Euryale could be an apt choice.



The goal here with Euryale is effectively to bring probabalistic decoding back to Stheno. Medusa does this using this whole attention thing, and their approach is certainly better. I created Euryale as a way to explore an alternative method I thought of that aims at doing largely the same thing but by taking advantage of batched inference rather than an attention mechanism. My hope is that this will only require more ram and not result in any added latency, but we'll see. 

The basic idea is to 
1) use topk instead of argmax
2) construct every single possible candidate sequence from the topk results of the medusa heads 
3) run every single one of those possible candidates through the model using batched inference
4) instead of checking against a greedy version of the NTP's output, check against the NTP's topk results
5) select whichever successful candidate is the longest. If there are multiple acceptable candidates, we now have to choose whether we want the one that was most probable, least probable, or somewhere in-between. There was no really clear way to do this; you can see what I settled on below

If anything i think the re-incorporation of topk results will *maybe* add a speed increase if they give us longer accepted sequences (although likely not as fast as the attention-based mechanism used in actual Medusa). However the real bottleneck I'm adding is memory. We're literally performing batched inference as opposed to single, and the check mechanism is also much larger. I'm not sure exactly how to do the calculation but I think the batched inference results in a linear increase in memory consumption on what was already the largest chunk of our computation, and then the topk matching results in a maybe exponential increase in memory usage but only with reference to a part that was already relatively small. Idk you decide,  i'm too tired at this point

In [26]:
# setup functions to make the actual loop less ugly

def combinations(tensor):
    '''
    takes our topk results and creates every possible permutation of them
    '''
    # Get the shape of the tensor
    m, k = tensor.shape

    # Create index grids for each dimension
    index_grids = [torch.arange(k) for _ in range(m)]

    # Create a meshgrid of indices with reverse order
    mesh_indices = torch.meshgrid(index_grids[::-1], indexing="ij")

    # Use advanced indexing to create the combination tensor
    # Reverse the order of tensor indexing to match the reversed mesh_indices
    combinations = torch.cat([tensor[m-1-i][mesh_indices[i]].unsqueeze(0) for i in range(m)], dim=0)

    # Reshape to the desired shape (k^m, m) and reverse the columns for the final output
    combinations = combinations.T.reshape(-1, m)
    combinations = combinations.flip(dims=[1])

    return combinations

def compare(A,B):
    '''
    compares our two ugly mess of candidate possibilities & checks. 
    Outputs a matrix that can be used to find the indices of interest
    '''
    i,j,k = A.shape
    
    match_tensor = (A == B).int() 
    print("match_tensor: ", match_tensor.shape, "\n", match_tensor)

    # We invert the tensor so that all zeros & 1's flip
    # Find the first 1 in each row. The max function returns the first occurrence of the maximum value.
    # We add one extra zero at the end of each row to handle rows that contain no zeros.
    pad = torch.ones((i,j,1), dtype=match_tensor.dtype, device=device)
    print("pad: ", pad.shape)#, "\n", pad)
    padded_tensor = torch.cat((1 - match_tensor, pad), dim=-1)
    print("padded_tensor: ", padded_tensor.shape, "\n", padded_tensor)
    zero_positions = padded_tensor.argmax(dim=-1)
    print("zero_positions: ", zero_positions.shape, "\n", zero_positions)

    # Adjust indices where the last position is selected (meaning there were no zeros)
    zero_positions[zero_positions >= k] = k
    print("zero_positions: ", zero_positions.shape, "\n", zero_positions)

    # Create a range tensor
    range_tensor = torch.arange(m, device=device).unsqueeze(0).expand_as(match_tensor)
    print("range_tensor: ", range_tensor.shape, "\n", range_tensor)

    # Create a mask where each element is 1 if it is before the first zero in its row
    mask = range_tensor < zero_positions.unsqueeze(-1)
    print("mask: ", mask.shape, "\n", mask)

    # Apply the mask and sum along each row
    result = (match_tensor * mask).sum(dim=-1)
    #print("result: ", result.shape)#, "\n", result)
    
    return result

In [27]:
def generate_Euryale(model, idx, max_runs, k=2):
    # Ensure idx is a single sequence
    assert idx.size(0) == 1, "idx must be of size (1, t)"
    print("idx: ", idx.shape)
    
    # crop idx to the last block_size tokens
    input_cond = idx[:, -t:] # should this be just t???? previosly i had it as -(t+m)
    print("idx_cond should be size (1, t): ", input_cond.shape)
    
    # get the predictions
    logits, loss, mlogits = model(input_cond) # (1,t,v), int, and (m,1,t,v)
    print("logits should be (1,t,v): ", logits.shape)
    print("mlogits should be (m,1,t,v): ", mlogits.shape)
    
    # medusa requires greedy decoding for the first regular t+1'th token
    # technically we could just do regular temperature probability sampling here BUT
    #   the problem is that would shift the distribution from what the medusa heads expect
    #   thereby giving us far worse results
    idx_ntp = torch.argmax(logits, dim=-1, keepdim=True).squeeze(dim=2) # (1, t)
    print("idx_ntp should be (1, t): ", idx_ntp.shape)
    
    # append sampled index to the running sequence
    idx = torch.cat((idx, idx_ntp[:,-1].unsqueeze(dim=0)), dim=1) # (1,t)+(1,1) -> (1, t+1)
    print("idx should be (1, t+1): ", idx.shape)

    # keep track of how many tokens we get per model inference
    tok_per_inf = [1]
    
    # we only want the medusa preditions for the newest future timestep
    mlogits = mlogits[...,-1,:] # becomes (m,1,v)
    mlogits = mlogits.squeeze(dim=1) # (m,v)
    print("mlogits should be (m,v): ", mlogits.shape)
    
    for _ in range(max_runs-1): # -1 since one iteration was done above
        
        # get the medusa heads' best guesses
        idx_m_topk = torch.topk(mlogits, k, dim=-1, largest=True).indices # (m,1,v) -> (m,k)
        print("idx_m_topk should be (m,k): ", idx_m_topk.shape)
        
        # so now we have idx shape (1,t+1) of confirmed tokens
        # and idx_m_topk shape (m,k) of topk speculative tokens from medusa heads
        # want to know whether model actually would've predicted any of the tokens 
        #   in idx_m so we need to run it with them & compare
        
        # get all the possible combinations of the topk results from each snake head
        mcomb = combinations(idx_m_topk) # (m, k) -> (k^m,m) of all k^m combinations
        print("mcomb should be (k^m,m): ", mcomb.shape, "\n", mcomb)
        
        # expanding out input sequence to take advantage of batched inference
        idx_rep = idx.repeat(k**m,1) # (1,t+1) -> (k^m, t+1)
        print("idx_rep should be (k^m,t+1): ", idx_rep.shape)
        
        # we construct a tensor composed of the initial context and then the speculative tokens
        # now this k^m dimension is effecitvely our batch size b as if we were doing batched inference
        inp = torch.cat((idx_rep, mcomb), dim=1) # (k^m,t+1) & (k^m,m) -> (k^m,t+1+m)
        print("input should be (k^m,t+1+m): ", inp.shape)
        
        # but since the model can't take more than t inputs
        # note: t is max content limit, but input might be shorter. if so that's fine
        input_cond = inp[:, -t:] 
        # (k^m,t+1+m) -> (k^m,t) where first t-1-m are prior context, the t-m'th is ntp, & last m are candidates
        print("input_cond should be (k^m,t): ", input_cond.shape, "\n", input_cond[:,-2*m:])
        
        # then we pass it in
        logits, loss, mlogits = model(input_cond) # (k^m,t,v), tensor of a single float, & (m,k^m,t,v)
        print("logits should be (k^m,t,v): ", logits.shape)
        print("mlogits should be (m,k^m,t,v): ", mlogits.shape)
        
        # instead of greedy or softmax & temperature, we grab topk values to be compared against mcomb later
        idx_ntp_topk = torch.topk(logits, k, dim=-1, largest=True).indices # (k^m,t,v) -> (k^m,t,k)
        print("idx_ntp_topk should be (k^m, t,k): ", idx_ntp_topk.shape, "\n", idx_ntp[:,-2*m:])
        
        # now for comparison's sake we need to ignore the indices that were part of the context
        # the regular ntp prediction can also be ignored
        # we just need to know the number of previous turn's medusa preds to keep
        idx_check = idx_ntp_topk[:,-(m+1):-1,:] # (k^m,t,k) -> (k^m,m,k)
        print("idx_check should be (k^m,m,k): ", idx_check.shape, "\n", idx_check)
        
        # get every possible combination to be checked against
        idx_check_comb = torch.stack([combinations(idx_check[i]) for i in range(idx_check.shape[0])])
        print("idx_check_comb should be (k^m,k^m,m): ", idx_check_comb.shape, "\n", idx_check_comb)
        
        # duplicate them so they can be compared to idx_check_comb
        mcomb_check = mcomb.unsqueeze(0).repeat(k**m,1,1) # (k^m,m) -> (1,k^m,m) -> (k^m,k^m,m)
        print("mcomb_check should be (k^m,k^m,m): ", mcomb_check.shape, "\n", mcomb_check)

        # check whether they match
        result = compare(mcomb_check,idx_check_comb)
        print("result:\n", result)
        
        # this is how many tokens our medusa head gave us
        max_val = torch.max(result).item()
        # keeping track
        tok_per_inf.append(max_val+1) # +1 for the extra NTP greedy token
        
        # however, there may be multiple possible candidates that give that number of tokens
        # the below rows look through 'result' and grab either the most or least likely rows or columns
        # this index flip works bc argmax always returns the index of the FIRST max value if there are multiple max values
        if random.choice([True, False]):
            max_idx_row = torch.max(result,1).indices[0].item() # most likely
            max_idx_col = len(result[max_idx_row]) - 1 - torch.argmax(result[max_idx_row].flip(0)).item() # least likely
        elif random.choice([True, False]):
            max_idx_row = (len(result) - 1 - torch.max(result.flip(dims=[0]),1).indices)[0].item() # least likely
            max_idx_col = torch.argmax(result[max_idx_row]).item() # most likely
        else:
            max_idx_row = torch.max(result,1).indices[0].item() # most likely
            max_idx_col = torch.argmax(result[max_idx_row]).item() # most likely
        print("max_val: ", max_val)
        print("max_idx_row: ", max_idx_row)
        print("max_idx_col: ", max_idx_col)
        # notice how those were not every possible combination, they're just the combination of probabilites I settled on
        # the problem with my approach is that if there are a lot of possibilities, only 3 of the options are made available
        # a more sophisticated approach would randomly choose between all the potential maximum candidates
        # actually now that I think about it, doing it that way might also be more computationally efficient
        # ugh i'm too tired tho idc anymore
        
        # so now i've got max_val which is an integer between 0 and m (inclusive)
        # and max_idx_row & max_idx_col which are indices of the best sequence in the k^m batch 
        # need to use them to get the actual medusa head outputs of interest and then append that to our sequence
        # bonus: we also get to use the regular ntp prediction right after any of last run's accepted medusa head predictions
        print("idx: ", idx.shape, idx)
        
        # now we splice out the chosen candidate sequence
        idx_m = mcomb[max_idx_row, :max_val].unsqueeze(0) # (k^m,m) -> (1,max_val)
        print("idx_m should be (1,max_val): ", idx_m.shape, "\n", idx_m)
        
        # and we also grab the free greedy token we get
        idx_ntp = torch.argmax(logits, dim=-1, keepdim=True).squeeze(dim=2) # (k^m,t,v) -> (k^m, t)
        print("idx_ntp should be (k^m,t): ", idx_ntp.shape, "\n", idx_ntp)
        idx_ntp = idx_ntp[max_idx_row, -1-m+max_val].unsqueeze(0).unsqueeze(0) # (k^m, t) -> (1,1)
        print("idx_ntp should be (1,1): ", idx_ntp.shape, "\n", idx_ntp)
        
        # and here we add them all together
        idx = torch.cat((idx, idx_m, idx_ntp),dim=1) # (1,t), (1,1), (1,max_val) -> (1,t+1+max_val)
        print("idx should be (1,t+max_val+1): ", idx.shape, "\n", idx)
        
        # wait but there's a problem. If we didn't use the largest max_val=m then 
        #     idx_m_topk is for items too far into the future. we need to pass forward 
        #     an idx_m_topk that's relevant
        # ngl i'm too tired to think about it so i'm assuming i can use the same index i used for idx_ntp
        mlogits = mlogits[:,max_idx_row,-1-m+max_val,:] # (m,k^m,t,v) -> (m,v)
        print("mlogits should be (m,v): ", mlogits.shape)
         
    return idx, tok_per_inf

In [28]:
%%time
input_str = "JULIET:\nO Romeo, Romeo! wherefore art thou R"
context_tensor = torch.tensor([encode(input_str)], dtype=torch.long, device=device)
output, tok_per_inf = generate_Euryale(model, context_tensor, max_runs=100, k=2)

idx:  torch.Size([1, 44])
idx_cond should be size (1, t):  torch.Size([1, 44])
logits should be (1,t,v):  torch.Size([1, 44, 65])
mlogits should be (m,1,t,v):  torch.Size([5, 1, 44, 65])
idx_ntp should be (1, t):  torch.Size([1, 44])
idx should be (1, t+1):  torch.Size([1, 45])
mlogits should be (m,v):  torch.Size([5, 65])
idx_m_topk should be (m,k):  torch.Size([5, 2])


/var/folders/4q/cm25yfx92119tgpbxz0xg4kh0000gn/T/ipykernel_59041/2442312487.py:16: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3618.)
  combinations = combinations.T.reshape(-1, m)


mcomb should be (k^m,m):  torch.Size([32, 5])
idx_rep should be (k^m,t+1):  torch.Size([32, 45])
input should be (k^m,t+1+m):  torch.Size([32, 50])
input_cond should be (k^m,t):  torch.Size([32, 50])
logits should be (k^m,t,v):  torch.Size([32, 50, 65])
mlogits should be (m,k^m,t,v):  torch.Size([5, 32, 50, 65])
idx_ntp_topk should be (k^m, t,k):  torch.Size([32, 50, 2])
idx_check should be (k^m,m,k):  torch.Size([32, 5, 2])
idx_check_comb should be (k^m,k^m,m):  torch.Size([32, 32, 5])
mcomb_check should be (k^m,k^m,m):  torch.Size([32, 32, 5])
match_tensor:  torch.Size([32, 32, 5])
pad:  torch.Size([32, 32, 1])
padded_tensor:  torch.Size([32, 32, 6])
zero_positions:  torch.Size([32, 32])
zero_positions:  torch.Size([32, 32])
range_tensor:  torch.Size([32, 32, 5])
mask:  torch.Size([32, 32, 5])
result:
 tensor([[2, 2, 2,  ..., 0, 0, 0],
        [2, 2, 2,  ..., 0, 0, 0],
        [2, 2, 2,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
 

input should be (k^m,t+1+m):  torch.Size([32, 64])
input_cond should be (k^m,t):  torch.Size([32, 64])
logits should be (k^m,t,v):  torch.Size([32, 64, 65])
mlogits should be (m,k^m,t,v):  torch.Size([5, 32, 64, 65])
idx_ntp_topk should be (k^m, t,k):  torch.Size([32, 64, 2])
idx_check should be (k^m,m,k):  torch.Size([32, 5, 2])
idx_check_comb should be (k^m,k^m,m):  torch.Size([32, 32, 5])
mcomb_check should be (k^m,k^m,m):  torch.Size([32, 32, 5])
match_tensor:  torch.Size([32, 32, 5])
pad:  torch.Size([32, 32, 1])
padded_tensor:  torch.Size([32, 32, 6])
zero_positions:  torch.Size([32, 32])
zero_positions:  torch.Size([32, 32])
range_tensor:  torch.Size([32, 32, 5])
mask:  torch.Size([32, 32, 5])
result:
 tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='mps:0')
max_val:  1
max_idx_row:  0
max_idx_col:  15
id

idx_rep should be (k^m,t+1):  torch.Size([32, 76])
input should be (k^m,t+1+m):  torch.Size([32, 81])
input_cond should be (k^m,t):  torch.Size([32, 81])
logits should be (k^m,t,v):  torch.Size([32, 81, 65])
mlogits should be (m,k^m,t,v):  torch.Size([5, 32, 81, 65])
idx_ntp_topk should be (k^m, t,k):  torch.Size([32, 81, 2])
idx_check should be (k^m,m,k):  torch.Size([32, 5, 2])
idx_check_comb should be (k^m,k^m,m):  torch.Size([32, 32, 5])
mcomb_check should be (k^m,k^m,m):  torch.Size([32, 32, 5])
match_tensor:  torch.Size([32, 32, 5])
pad:  torch.Size([32, 32, 1])
padded_tensor:  torch.Size([32, 32, 6])
zero_positions:  torch.Size([32, 32])
zero_positions:  torch.Size([32, 32])
range_tensor:  torch.Size([32, 32, 5])
mask:  torch.Size([32, 32, 5])
result:
 tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], device='mps:

...

idx_check_comb should be (k^m,k^m,m):  torch.Size([32, 32, 5])
mcomb_check should be (k^m,k^m,m):  torch.Size([32, 32, 5])
match_tensor:  torch.Size([32, 32, 5])
pad:  torch.Size([32, 32, 1])
padded_tensor:  torch.Size([32, 32, 6])
zero_positions:  torch.Size([32, 32])
zero_positions:  torch.Size([32, 32])
range_tensor:  torch.Size([32, 32, 5])
mask:  torch.Size([32, 32, 5])
result:
 tensor([[2, 2, 2,  ..., 1, 1, 1],
        [2, 2, 2,  ..., 1, 1, 1],
        [2, 2, 2,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]], device='mps:0')
max_val:  2
max_idx_row:  31
max_idx_col:  0
idx:  torch.Size([1, 274]) tensor([[22, 33, 24, 21, 17, 32, 10,  0, 27,  1, 30, 53, 51, 43, 53,  6,  1, 30,
         53, 51, 43, 53,  2,  1, 61, 46, 43, 56, 43, 44, 53, 56, 43,  1, 39, 56,
         58,  1, 58, 46, 53, 59,  1, 30, 53, 51, 43, 12,  0,  0,  0, 24, 27, 30,
         16,  1,  0, 24, 33, 15, 21, 27, 10,  0, 27,  6,  1, 46,

logits should be (k^m,t,v):  torch.Size([32, 128, 65])
mlogits should be (m,k^m,t,v):  torch.Size([5, 32, 128, 65])
idx_ntp_topk should be (k^m, t,k):  torch.Size([32, 128, 2])
idx_check should be (k^m,m,k):  torch.Size([32, 5, 2])
idx_check_comb should be (k^m,k^m,m):  torch.Size([32, 32, 5])
mcomb_check should be (k^m,k^m,m):  torch.Size([32, 32, 5])
match_tensor:  torch.Size([32, 32, 5])
pad:  torch.Size([32, 32, 1])
padded_tensor:  torch.Size([32, 32, 6])
zero_positions:  torch.Size([32, 32])
zero_positions:  torch.Size([32, 32])
range_tensor:  torch.Size([32, 32, 5])
mask:  torch.Size([32, 32, 5])
result:
 tensor([[2, 2, 2,  ..., 0, 0, 0],
        [2, 2, 2,  ..., 0, 0, 0],
        [2, 2, 2,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='mps:0')
max_val:  3
max_idx_row:  0
max_idx_col:  7
idx:  torch.Size([1, 283]) tensor([[22, 33, 24, 21, 17, 32, 10,  0, 27,  1, 30, 53, 51, 43, 53,  6,  1,

logits should be (k^m,t,v):  torch.Size([32, 128, 65])
mlogits should be (m,k^m,t,v):  torch.Size([5, 32, 128, 65])
idx_ntp_topk should be (k^m, t,k):  torch.Size([32, 128, 2])
idx_check should be (k^m,m,k):  torch.Size([32, 5, 2])
idx_check_comb should be (k^m,k^m,m):  torch.Size([32, 32, 5])
mcomb_check should be (k^m,k^m,m):  torch.Size([32, 32, 5])
match_tensor:  torch.Size([32, 32, 5])
pad:  torch.Size([32, 32, 1])
padded_tensor:  torch.Size([32, 32, 6])
zero_positions:  torch.Size([32, 32])
zero_positions:  torch.Size([32, 32])
range_tensor:  torch.Size([32, 32, 5])
mask:  torch.Size([32, 32, 5])
result:
 tensor([[2, 2, 2,  ..., 0, 0, 0],
        [2, 2, 2,  ..., 0, 0, 0],
        [2, 2, 2,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='mps:0')
max_val:  2
max_idx_row:  31
max_idx_col:  0
idx:  torch.Size([1, 293]) tensor([[22, 33, 24, 21, 17, 32, 10,  0, 27,  1, 30, 53, 51, 43, 53,  6,  1

logits should be (k^m,t,v):  torch.Size([32, 128, 65])
mlogits should be (m,k^m,t,v):  torch.Size([5, 32, 128, 65])
idx_ntp_topk should be (k^m, t,k):  torch.Size([32, 128, 2])
idx_check should be (k^m,m,k):  torch.Size([32, 5, 2])
idx_check_comb should be (k^m,k^m,m):  torch.Size([32, 32, 5])
mcomb_check should be (k^m,k^m,m):  torch.Size([32, 32, 5])
match_tensor:  torch.Size([32, 32, 5])
pad:  torch.Size([32, 32, 1])
padded_tensor:  torch.Size([32, 32, 6])
zero_positions:  torch.Size([32, 32])
zero_positions:  torch.Size([32, 32])
range_tensor:  torch.Size([32, 32, 5])
mask:  torch.Size([32, 32, 5])
result:
 tensor([[2, 2, 2,  ..., 0, 0, 0],
        [2, 2, 2,  ..., 0, 0, 0],
        [2, 2, 2,  ..., 0, 0, 0],
        ...,
        [2, 2, 2,  ..., 0, 0, 0],
        [2, 2, 2,  ..., 0, 0, 0],
        [2, 2, 2,  ..., 0, 0, 0]], device='mps:0')
max_val:  2
max_idx_row:  0
max_idx_col:  7
idx:  torch.Size([1, 299]) tensor([[22, 33, 24, 21, 17, 32, 10,  0, 27,  1, 30, 53, 51, 43, 53,  6,  1,

logits should be (k^m,t,v):  torch.Size([32, 128, 65])
mlogits should be (m,k^m,t,v):  torch.Size([5, 32, 128, 65])
idx_ntp_topk should be (k^m, t,k):  torch.Size([32, 128, 2])
idx_check should be (k^m,m,k):  torch.Size([32, 5, 2])
idx_check_comb should be (k^m,k^m,m):  torch.Size([32, 32, 5])
mcomb_check should be (k^m,k^m,m):  torch.Size([32, 32, 5])
match_tensor:  torch.Size([32, 32, 5])
pad:  torch.Size([32, 32, 1])
padded_tensor:  torch.Size([32, 32, 6])
zero_positions:  torch.Size([32, 32])
zero_positions:  torch.Size([32, 32])
range_tensor:  torch.Size([32, 32, 5])
mask:  torch.Size([32, 32, 5])
result:
 tensor([[2, 2, 2,  ..., 0, 0, 0],
        [2, 2, 2,  ..., 0, 0, 0],
        [2, 2, 2,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='mps:0')
max_val:  3
max_idx_row:  0
max_idx_col:  15
idx:  torch.Size([1, 308]) tensor([[22, 33, 24, 21, 17, 32, 10,  0, 27,  1, 30, 53, 51, 43, 53,  6,  1

In [29]:
print("tokens per inference: ", sum(tok_per_inf)/len(tok_per_inf))
print(decode(output[0].tolist()))

tokens per inference:  2.69
JULIET:
O Romeo, Romeo! wherefore art thou Rome?


LORD 
LUCIO:
O, heweep  the man of    toreee than  ofriend,
An ihoed the strenget of the strekge oo eart
Th    the staeeee of the stree  of the strenge

First Servinr: are tou thie?

Clown:
What  is,moe?
WMaster the   trualo ooa that  we haav 
shall both the  se
